In [7]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#adaptacion
from mSDA import msda
from utils.adaptacion import adapt_msda

#otros
import numpy as np
import pandas as pd

tipo = pruebas[1]
dataset_name = dataset[1]
dims = dimensions[dataset_name]

## Pruebas con el dataset Twitter

In [2]:
print tipo
print dataset_name
print data_path

msda
twitter
data


In [3]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [4]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [5]:
pr = 0.8
n_layers = 5

# Se entrena una sola vez con todos los datos disponibles
print "Adaptando dominios..."
t_adaptar, train_mappings = adapt_msda(X[:, :dims], pr=pr, n_layers=n_layers)
print "Adaptacion realizada en %.3f segundos." % t_adaptar

Adaptando dominios...
Adaptacion realizada en 23.687 segundos.


In [8]:
#diccionario para mantener los dominios adaptados
adapted = {}

print "Transformando dominios a una nueva representacion..."
for domain in domains:
    print "Adaptando %s" % domain
    X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
    X_ts = np.asarray(labeled[domain]['X_ts'][:, :dims].todense())
    
    tr_mappings, tr_reps = msda.mSDA(X_tr[:, :dims], pr, n_layers, train_mappings)
    ts_mappings, ts_reps = msda.mSDA(X_ts[:, :dims], pr, n_layers, train_mappings)
    
    adapted[domain] = {
        'X_tr': tr_reps[-1],
        'X_ts': ts_reps[-1]
    }
    
print "Dominios transformados."

Transformando dominios a una nueva representacion...
Adaptando rio2016
Adaptando thevoice
Dominios transformados.


In [9]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1

print "Pruebas completadas"

Tarea 1 de 2
Cargando modelo existente.
Tarea 2 de 2
Cargando modelo existente.
Pruebas completadas


In [10]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,r->t,rio2016,thevoice,16.081594,50.769231,34.687637
1,mSDA,t->r,thevoice,rio2016,20.687361,40.625000,19.937639


In [11]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/msda/twitter.csv
Resultados guardados.
